In [1]:
import ginza # pip install -U ginza ja_ginza
import nltk  # pip install nltk
import spacy # pip install nltk
import pke   # pip install git+https://github.com/boudinfl/pke.git
import pandas as pd

In [2]:
df = pd.read_csv('datasets/bachelor_themes.csv')
df.head()

,年度,学科／研究科,研究室,学籍番号,賞,氏名,要旨,題目
0,H17,情報科学科,相研,NaN,NaN,NaN,NaN,WEBページにおけるユーザビリティの追及
1,H17,情報科学科,松永研,NaN,NaN,NaN,NaN,安全運転管理教育システム(ASSIST)における複数カメラ使用での交通事故防止対策
2,H17,情報科学科,宮崎研,NaN,NaN,NaN,NaN,電子透かしの体制に関する検討 -加重平均フィルタ・メディアンフィルタの場合-
3,H17,情報科学科,成研,NaN,NaN,NaN,NaN,対面教育を支援するためのウェブ助言システム
4,H17,情報科学科,石田健研,NaN,NaN,NaN,NaN,衛星通信検討のための気象データの解析-2002年のデータ解析-


In [3]:
pke.lang.stopwords['ja_ginza'] = 'ja'
def get_key_phrase(spacy_model, text, n = 5, stopwords=None):
    """キーフレーズ処理

    Args:
        text ([type]): キーフレーズ対象の文字列
        n (int, optional): [description]. Defaults to 10.
    Returns:
        [type]: [description]
    """
    #MultipartiteRankなどのキーフレーズの手法については以下のサイトが参考になります
    #https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part5.html
    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(input=text, language='ja', normalization=None, stoplist=stopwords)
    extractor.candidate_selection( pos={'NOUN', 'PROPN', 'ADJ', 'NUM'})
    extractor.candidate_weighting(threshold=0.10, method='average', alpha=1.1)
    key_phrase = extractor.get_n_best(n)
    
    return  key_phrase

In [4]:
from  spacy.lang.ja import stop_words
spacy_model = spacy.load("ja_ginza")  # GiNZAモデルの読み込み
stopwords = list(stop_words.STOP_WORDS)
print(stopwords[-10:])
nltk.corpus.stopwords.words = stopwords

['せ', 'いる', 'ぶり', 'すぐ', 'あるいは', 'なお', 'きっかけ', 'ごと', 'つつ', 'それ']


In [5]:
get_key_phrase(spacy_model,u"電子透かしの体制に関する検討 -加重平均フィルタ・メディアンフィルタの場合",stopwords=stopwords)

[('加重平均 フィルタ', 0.42130324238360667),
 ('メディアン フィルタ', 0.39338731541367405),
 ('電子 透かし', 0.18530944220271967)]

In [6]:
get_key_phrase(spacy_model,u"人の反応・判断能力測定 ドライビングシミュレータ",stopwords=stopwords)

[('判断 能力 測定 ドライビング シミュレータ', 1.0)]

In [7]:
from pprint import pprint
titles =  [t[0] for t in df.loc[:,['題目']].values]
for text in titles[:30]:
    kp = get_key_phrase(spacy_model,text, stopwords=stopwords)
    print(text)
    pprint(kp)
    print("*"*10)

WEBページにおけるユーザビリティの追及
[('webページ', 0.5), ('ユーザビリティ', 0.5)]
**********
安全運転管理教育システム(ASSIST)における複数カメラ使用での交通事故防止対策
[('assist', 0.28706225950104136),
 ('安全 運転 管理 教育 システム', 0.2624476088748522),
 ('複数 カメラ 使用', 0.2515057737576961),
 ('交通事故 防止 対策', 0.19898435786641003)]
**********
電子透かしの体制に関する検討 -加重平均フィルタ・メディアンフィルタの場合-
[('加重平均 フィルタ', 0.42130324238360667),
 ('メディアン フィルタ', 0.39338731541367405),
 ('電子 透かし', 0.18530944220271967)]
**********
対面教育を支援するためのウェブ助言システム
[('対面 教育', 0.5), ('ウェブ 助言 システム', 0.5)]
**********
衛星通信検討のための気象データの解析-2002年のデータ解析-
[('気象 データ', 0.40455778257937486),
 ('衛星 通信 検討', 0.3467191085775726),
 ('データ 解析', 0.24872310884305232)]
**********
手書き漢字認識装置の性能調査
[('手書き 漢字 認識 装置', 0.5), ('性能 調査', 0.5)]
**********
生化学反応シミュレータの比較評価
[('生化学 反応 シミュレータ', 0.5), ('比較 評価', 0.5)]
**********
FPGAを搭載したロボット教材のためのIPとしてKUE-CHIP2の検討
[('ロボット 教材', 0.4023301515526182),
 ('fpga', 0.34007062296461077),
 ('kue', 0.2575992254827705)]
**********
手の輪郭を用いた生体認証に関する研究
[('生体 認証', 1.0)]
**********
重畳積文法によるX線CTの画像再構成-Shepp-